# Model Training Pipeline

This notebook demonstrates the complete model training pipeline including:
- Loading preprocessed data
- Training multiple ML algorithms
- Hyperparameter tuning and optimization
- Model evaluation and comparison
- Performance visualization and analysis
- Model selection and persistence

**Author:** MLOps Team  
**Version:** 1.0.0  
**Date:** November 2024

## 1. Setup and Configuration

Import necessary libraries and configure the training environment.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime

# Standard ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle
import json

# Configure warnings and display
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully")
print(f"📅 Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Basic configuration setup
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Define paths
DATA_DIR = Path("../data")
PROCESSED_DATA_DIR = DATA_DIR / "processed"
MODELS_DIR = Path("../models")
MODELS_DIR.mkdir(exist_ok=True)

print(f"🔧 Configuration setup complete")
print(f"🎲 Random state: {RANDOM_STATE}")
print(f"📂 Data directory: {PROCESSED_DATA_DIR}")
print(f"💾 Models directory: {MODELS_DIR}")

# Verify processed data exists
if not PROCESSED_DATA_DIR.exists():
    print("❌ Processed data directory not found!")
    print("Please run the data processing pipeline first (02_data_processing_pipeline.ipynb)")
else:
    print("✅ Processed data directory found")

print("✅ Model training environment configured")

## 2. Load Preprocessed Data

Load the training and test datasets that were prepared in the data processing pipeline.

In [ ]:
# Load processed datasets
train_file = PROCESSED_DATA_DIR / "iris_train.csv"
test_file = PROCESSED_DATA_DIR / "iris_test.csv"
metadata_file = PROCESSED_DATA_DIR / "preprocessing_metadata.pkl"

if not train_file.exists() or not test_file.exists():
    print("❌ Processed data files not found!")
    print("Please run the data processing pipeline first (02_data_processing_pipeline.ipynb)")
    raise FileNotFoundError("Processed data files missing")

# Load training and test data
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

print(f"📊 Training data shape: {train_data.shape}")
print(f"📊 Test data shape: {test_data.shape}")

# Display first few rows
print("\n🔍 Training data preview:")
print(train_data.head())

# Check for missing values
print("\n🔍 Missing values check:")
print(f"Training data: {train_data.isnull().sum().sum()} missing values")
print(f"Test data: {test_data.isnull().sum().sum()} missing values")

print(f"\n✅ Loaded training data: {train_data.shape}, test data: {test_data.shape}")

In [ ]:
# Separate features and targets
target_column = 'target'  # Our processed data uses 'target' column

# Training data
X_train = train_data.drop(columns=[target_column])
y_train = train_data[target_column]

# Test data
X_test = test_data.drop(columns=[target_column])
y_test = test_data[target_column]

print(f"🎯 Features shape - Train: {X_train.shape}, Test: {X_test.shape}")
print(f"🎯 Target shape - Train: {y_train.shape}, Test: {y_test.shape}")

# Display feature information
print("\n📋 Feature columns:")
for i, col in enumerate(X_train.columns):
    print(f"{i+1}. {col}")

# Display target classes
unique_classes = sorted(y_train.unique())
print(f"\n🏷️ Target classes: {unique_classes}")
print(f"🔢 Number of classes: {len(unique_classes)}")

# Class distribution
print("\n📊 Class distribution in training data:")
class_counts = y_train.value_counts().sort_index()
for class_idx, count in class_counts.items():
    percentage = (count / len(y_train)) * 100
    # Map numeric classes to iris species
    class_names = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
    class_name = class_names.get(class_idx, f'class_{class_idx}')
    print(f"  {class_name} (class {class_idx}): {count} samples ({percentage:.1f}%)")

print(f"\n✅ Prepared features and targets - Classes: {unique_classes}")

## 3. Data Exploration and Visualization

Understand the data distribution and relationships before training models.

In [ ]:
# Basic statistics for features
print("📈 Feature Statistics:")
print(X_train.describe())

# Feature correlations
plt.figure(figsize=(10, 8))
correlation_matrix = X_train.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔍 Highly correlated feature pairs (|correlation| > 0.7):")
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            feature1 = correlation_matrix.columns[i]
            feature2 = correlation_matrix.columns[j]
            high_corr_pairs.append((feature1, feature2, corr_val))
            print(f"  {feature1} ↔ {feature2}: {corr_val:.3f}")

if not high_corr_pairs:
    print("  No highly correlated feature pairs found.")

print("✅ Completed basic data exploration")

In [ ]:
# Feature distributions by class
n_features = len(X_train.columns)
n_cols = 2
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
if n_rows > 1:
    axes = axes.ravel()
else:
    axes = [axes] if n_cols == 1 else axes

for i, feature in enumerate(X_train.columns):
    ax = axes[i]
    
    # Create combined data for plotting
    combined_data = pd.DataFrame({
        feature: X_train[feature],
        'target': y_train
    })
    
    # Box plot by class
    sns.boxplot(data=combined_data, x='target', y=feature, ax=ax)
    ax.set_title(f'Distribution of {feature} by Class', fontweight='bold')
    ax.set_xlabel('Class')
    
    # Add class labels
    ax.set_xticklabels(['setosa', 'versicolor', 'virginica'])

# Remove empty subplots
for i in range(n_features, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

print("✅ Visualized feature distributions by class")

## 4. Model Training and Evaluation

Train multiple machine learning models and compare their performance.

In [ ]:
# Define model configurations
model_configs = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=RANDOM_STATE),
        'param_grid': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'Logistic Regression': {
        'model': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
        'param_grid': {
            'C': [0.1, 1.0, 10.0, 100.0],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    'SVM': {
        'model': SVC(random_state=RANDOM_STATE, probability=True),
        'param_grid': {
            'C': [0.1, 1.0, 10.0],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['scale', 'auto']
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(random_state=RANDOM_STATE),
        'param_grid': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7]
        }
    }
}

print("🔧 Model Configurations:")
print("=" * 50)

for model_name, config in model_configs.items():
    print(f"\n📋 {model_name}")
    print(f"   Algorithm: {config['model'].__class__.__name__}")
    print(f"   Hyperparameters to tune: {list(config['param_grid'].keys())}")

print(f"\n📊 Total models configured: {len(model_configs)}")
print("✅ Model configurations ready")

In [ ]:
# Train all models with hyperparameter tuning
print("🚀 Starting model training with hyperparameter tuning...")
print("This may take several minutes depending on the parameter grids.")
print("=" * 60)

start_time = datetime.now()
training_results = {}

for model_name, config in model_configs.items():
    print(f"\n🔄 Training {model_name}...")
    model_start_time = datetime.now()
    
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(
        estimator=config['model'],
        param_grid=config['param_grid'],
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        verbose=0
    )
    
    # Fit the model
    grid_search.fit(X_train, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_
    
    # Make predictions
    train_predictions = best_model.predict(X_train)
    test_predictions = best_model.predict(X_test)
    
    # Calculate metrics
    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    # Cross-validation scores
    cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
    
    # Additional metrics
    precision = precision_score(y_test, test_predictions, average='weighted')
    recall = recall_score(y_test, test_predictions, average='weighted')
    f1 = f1_score(y_test, test_predictions, average='weighted')
    
    # Training time
    model_end_time = datetime.now()
    training_time = (model_end_time - model_start_time).total_seconds()
    
    # Store results
    training_results[model_name] = {
        'model': best_model,
        'best_params': grid_search.best_params_,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'training_time': training_time,
        'test_predictions': test_predictions
    }
    
    print(f"   ✅ {model_name} completed in {training_time:.2f}s - Accuracy: {test_accuracy:.4f}")

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

print("\n✅ Model training completed!")
print("=" * 60)
print(f"⏱️  Total training time: {total_time:.2f} seconds")
print(f"🏆 Models trained successfully: {len(training_results)}")

# Display quick results summary
print("\n📊 Quick Results Summary:")
for model_name, result in training_results.items():
    print(f"  {model_name}: {result['test_accuracy']:.4f} accuracy")

print(f"\n✅ Successfully trained {len(training_results)} models in {total_time:.2f}s")

# Create comprehensive model comparison
comparison_data = []
for model_name, result in training_results.items():
    comparison_data.append({
        'Model': model_name,
        'Train_Accuracy': result['train_accuracy'],
        'Test_Accuracy': result['test_accuracy'],
        'CV_Mean': result['cv_mean'],
        'CV_Std': result['cv_std'],
        'Precision': result['precision'],
        'Recall': result['recall'],
        'F1_Score': result['f1_score'],
        'Training_Time': result['training_time']
    })

comparison_df = pd.DataFrame(comparison_data)

print("📊 Model Performance Comparison:")
print("=" * 80)

# Display with nice formatting
print(comparison_df.round(4))

# Identify best performing models
best_test_acc = comparison_df.loc[comparison_df['Test_Accuracy'].idxmax()]
best_cv = comparison_df.loc[comparison_df['CV_Mean'].idxmax()]
best_f1 = comparison_df.loc[comparison_df['F1_Score'].idxmax()]

print("\n🏆 Top Performers:")
print(f"  🎯 Best Test Accuracy: {best_test_acc['Model']} ({best_test_acc['Test_Accuracy']:.4f})")
print(f"  📈 Best Cross-Validation: {best_cv['Model']} ({best_cv['CV_Mean']:.4f})")
print(f"  ⚖️ Best F1-Score: {best_f1['Model']} ({best_f1['F1_Score']:.4f})")

# Get the best overall model (highest test accuracy)
best_model_name = best_test_acc['Model']
best_result = training_results[best_model_name]

print(f"\n🏆 Selected Best Model: {best_model_name}")
print(f"✅ Best model by test accuracy: {best_model_name} ({best_test_acc['Test_Accuracy']:.4f})")

In [ ]:
# Create comprehensive model comparison
comparison_data = []
for model_name, result in training_results.items():
    comparison_data.append({
        'Model': model_name,
        'Train_Accuracy': result['train_accuracy'],
        'Test_Accuracy': result['test_accuracy'],
        'CV_Mean': result['cv_mean'],
        'CV_Std': result['cv_std'],
        'Precision': result['precision'],
        'Recall': result['recall'],
        'F1_Score': result['f1_score'],
        'Training_Time': result['training_time']
    })

comparison_df = pd.DataFrame(comparison_data)

print("📊 Model Performance Comparison:")
print("=" * 80)
print(comparison_df.round(4))

# Identify best performing models
best_test_acc = comparison_df.loc[comparison_df['Test_Accuracy'].idxmax()]
best_cv = comparison_df.loc[comparison_df['CV_Mean'].idxmax()]
best_f1 = comparison_df.loc[comparison_df['F1_Score'].idxmax()]

print("\n🏆 Top Performers:")
print(f"  🎯 Best Test Accuracy: {best_test_acc['Model']} ({best_test_acc['Test_Accuracy']:.4f})")
print(f"  📈 Best Cross-Validation: {best_cv['Model']} ({best_cv['CV_Mean']:.4f})")
print(f"  ⚖️ Best F1-Score: {best_f1['Model']} ({best_f1['F1_Score']:.4f})")

# Get the best overall model (highest test accuracy)
best_model_name = best_test_acc['Model']
best_result = training_results[best_model_name]
best_model = training_results[best_model_name]['model']

print(f"\n🏆 Selected Best Model: {best_model_name}")

# Detailed results for best model
print(f"\n📋 Best Model Details:")
print(f"   🎯 Test Accuracy: {best_result['test_accuracy']:.4f}")
print(f"   📈 Cross-Validation: {best_result['cv_mean']:.4f} ± {best_result['cv_std']:.4f}")
print(f"   🔧 Best Parameters: {best_result['best_params']}")
print(f"   ⏱️  Training Time: {best_result['training_time']:.2f}s")

## 5. Model Evaluation and Visualization

Detailed analysis of model performance with visualizations.

In [ ]:
# Create model comparison visualization
print("📈 Creating model performance comparison plot...")

# Prepare data for visualization
metrics_map = {
    'Test Accuracy': 'test_accuracy',
    'Precision': 'precision', 
    'Recall': 'recall',
    'F1 Score': 'f1_score'
}

models = list(training_results.keys())

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, (metric_name, metric_key) in enumerate(metrics_map.items()):
    ax = axes[i]
    
    # Get metric values
    values = [training_results[model][metric_key] for model in models]
    colors = ['gold' if model == best_model_name else 'skyblue' for model in models]
    
    # Create bar plot
    bars = ax.bar(models, values, color=colors, alpha=0.8, edgecolor='black')
    ax.set_title(f'Model Comparison - {metric_name}', fontweight='bold')
    ax.set_ylabel(metric_name)
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Set y-axis limits for better visualization
    ax.set_ylim(0, 1.1)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("✅ Generated model comparison visualization")

In [ ]:
# Debug: Check the actual structure of training_results
print("Debug - Training Results Structure:")
for model_name, result in training_results.items():
    print(f"\n{model_name}:")
    for key, value in result.items():
        print(f"  {key}: {value}")

print(f"\nBest model: {best_model_name}")
print(f"Best result keys: {list(best_result.keys())}")

In [ ]:
# Feature importance plots for models that support it
print("🔍 Creating feature importance plots...")

# Only include models that have feature importance
models_with_importance = {}
for name, result in training_results.items():
    if 'feature_importance' in result and result['feature_importance'] is not None:
        models_with_importance[name] = result

if models_with_importance:
    n_models = len(models_with_importance)
    n_cols = 2
    n_rows = (n_models + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    if n_rows == 1:
        axes = [axes] if n_models == 1 else axes
    else:
        axes = axes.ravel()
    
    for idx, (model_name, result) in enumerate(models_with_importance.items()):
        ax = axes[idx] if n_models > 1 else axes
        
        # Get feature importance from dictionary
        feature_importance = result['feature_importance']
        features = list(feature_importance.keys())
        importance_values = list(feature_importance.values())
        
        # Create horizontal bar plot
        y_pos = np.arange(len(features))
        ax.barh(y_pos, importance_values, alpha=0.8, color='skyblue')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(features)
        ax.invert_yaxis()
        ax.set_xlabel('Importance Score')
        ax.set_title(f'Feature Importance - {model_name}', fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
    
    # Remove empty subplots
    for idx in range(len(models_with_importance), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Feature importance plots created for {len(models_with_importance)} models")
else:
    print("ℹ️  No models with feature importance available")

print("✅ Feature importance analysis completed")

## 7. Model Selection and Analysis

Select the best model and perform detailed analysis.

In [ ]:
# Detailed analysis of the best model
print("Best Model Analysis:")
print("=" * 50)

print(f"\nSelected Model: {best_model_name}")
print(f"Test Accuracy: {best_result['test_accuracy']:.4f}")
print(f"Training Accuracy: {best_result['train_accuracy']:.4f}")
print(f"Cross-Validation Score: {best_result['cv_mean']:.4f} ± {best_result['cv_std']:.4f}")
print(f"Training Time: {best_result['training_time']:.2f} seconds")

# Performance metrics
print(f"\nDetailed Performance Metrics:")
print(f"  • Precision: {best_result['precision']:.4f}")
print(f"  • Recall: {best_result['recall']:.4f}")
print(f"  • F1-Score: {best_result['f1_score']:.4f}")

# Best hyperparameters
print(f"\nOptimized Hyperparameters:")
for param, value in best_result['best_params'].items():
    print(f"  {param}: {value}")

# Performance insights
print(f"\nPerformance Insights:")
accuracy_gap = abs(best_result['train_accuracy'] - best_result['test_accuracy'])
print(f"  • Training vs Test Accuracy Gap: {accuracy_gap:.4f}")
if accuracy_gap > 0.05:
    print(f"    WARNING: Large gap may indicate overfitting")
else:
    print(f"    GOOD: Good generalization capability")

print(f"  • CV Standard Deviation: {best_result['cv_std']:.4f}")
if best_result['cv_std'] > 0.05:
    print(f"    WARNING: High variance across CV folds")
else:
    print(f"    GOOD: Stable performance across folds")

print(f"\nModel Ranking by Test Accuracy:")
# Sort all models by test accuracy
sorted_models = sorted(training_results.items(), 
                      key=lambda x: x[1]['test_accuracy'], 
                      reverse=True)
for i, (name, result) in enumerate(sorted_models, 1):
    print(f"  {i}. {name}: {result['test_accuracy']:.4f}")

print("\nBest model analysis completed")

In [ ]:
# Show some prediction examples
print("Prediction Examples (Best Model):")
print("=" * 50)

# Make predictions with the best model
best_model_predictions = best_model.predict(X_test)
best_model_probabilities = None

if hasattr(best_model, 'predict_proba'):
    best_model_probabilities = best_model.predict_proba(X_test)

# Show first 10 predictions
print("\nFirst 10 Test Predictions:")
print(f"{'Index':<6} {'Actual':<12} {'Predicted':<12} {'Correct':<8}")
print("-" * 45)

for i in range(min(10, len(X_test))):
    actual = y_test.iloc[i]
    predicted = best_model_predictions[i]
    is_correct = "YES" if actual == predicted else "NO"
    
    print(f"{i:<6} {actual:<12} {predicted:<12} {is_correct:<8}")

# Accuracy breakdown by class
print("\nAccuracy by Class:")
from sklearn.metrics import classification_report
class_report = classification_report(y_test, best_model_predictions, output_dict=True)

for class_name in sorted(y_test.unique()):
    if str(class_name) in class_report:
        precision = class_report[str(class_name)]['precision']
        recall = class_report[str(class_name)]['recall']
        f1 = class_report[str(class_name)]['f1-score']
        support = class_report[str(class_name)]['support']
        
        print(f"  Class {class_name}:")
        print(f"    Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f} ({int(support)} samples)")

print("\nPrediction examples completed")

In [ ]:
# Save the best model and create metadata
print("Saving best model and creating metadata...")

# Create timestamp for file naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Define file paths
best_model_path = MODELS_DIR / f"best_model_{timestamp}.pkl"
all_models_path = MODELS_DIR / f"all_models_{timestamp}.pkl"
results_summary_path = MODELS_DIR / f"training_results_{timestamp}.pkl"

# Save the best model
with open(best_model_path, 'wb') as f:
    pickle.dump(best_model, f)

# Save all models
all_models_dict = {name: result['model'] for name, result in training_results.items()}
with open(all_models_path, 'wb') as f:
    pickle.dump(all_models_dict, f)

# Save training results
with open(results_summary_path, 'wb') as f:
    pickle.dump(training_results, f)

# Create model metadata file
model_metadata = {
    'model_info': {
        'name': best_model_name,
        'algorithm': best_model_name,  # Using model name as algorithm
        'version': '1.0.0',
        'training_date': datetime.now().isoformat(),
        'random_state': int(RANDOM_STATE)
    },
    'performance': {
        'test_accuracy': float(best_result['test_accuracy']),
        'cv_mean': float(best_result['cv_mean']),
        'cv_std': float(best_result['cv_std']),
        'precision': float(best_result['precision']),
        'recall': float(best_result['recall']),
        'f1_score': float(best_result['f1_score']),
        'training_time': float(best_result['training_time'])
    },
    'data_info': {
        'training_samples': int(len(X_train)),
        'test_samples': int(len(X_test)),
        'features': list(X_train.columns),
        'target_classes': [int(x) for x in sorted(y_train.unique())],
        'feature_count': int(len(X_train.columns)),
        'class_count': int(len(y_train.unique()))
    },
    'hyperparameters': best_result['best_params'],
    'files': {
        'model_file': str(best_model_path),
        'results_file': str(results_summary_path),
        'all_models_file': str(all_models_path)
    }
}

# Save metadata
metadata_path = MODELS_DIR / f"model_metadata_{timestamp}.json"
import json
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)

print(f"Model metadata saved: {metadata_path}")

# Display metadata summary
print("\nModel Deployment Information:")
print(f"  Model Name: {model_metadata['model_info']['name']}")
print(f"  Algorithm: {model_metadata['model_info']['algorithm']}")
print(f"  Test Accuracy: {model_metadata['performance']['test_accuracy']:.4f}")
print(f"  Features: {model_metadata['data_info']['feature_count']}")
print(f"  Classes: {model_metadata['data_info']['class_count']}")
print(f"  Model File: {best_model_path.name}")

print("\nModel saving completed")

# Create comprehensive evaluation report
evaluation_dir = Path("../evaluation_reports")

print("📄 Creating comprehensive evaluation report...")
print("This will include all visualizations and detailed analysis.")

report_path = evaluator.create_evaluation_report(
    training_results,
    output_dir=str(evaluation_dir)
)

print(f"\n✅ Comprehensive evaluation report created!")
print(f"📁 Report location: {report_path}")
print(f"📁 Report directory: {evaluation_dir}")

# List generated files
if evaluation_dir.exists():
    report_files = list(evaluation_dir.glob("*"))
    print(f"\n📋 Generated files ({len(report_files)}):")
    for file_path in sorted(report_files):
        file_size = file_path.stat().st_size / 1024  # KB
        print(f"  📄 {file_path.name} ({file_size:.1f} KB)")

logger.info(f"Comprehensive evaluation report generated: {report_path}")

# Final training summary
print("🎯 MODEL TRAINING PIPELINE SUMMARY")
print("=" * 60)

end_time = datetime.now()
notebook_duration = (end_time - start_time).total_seconds()

print(f"\n⏱️  Pipeline Duration: {notebook_duration:.1f} seconds")
print(f"📊 Total Models Trained: {len(training_results)}")
print(f"🏆 Best Model: {best_model_name}")
print(f"🎯 Best Accuracy: {best_result.test_accuracy:.4f}")
print(f"📈 Cross-Validation Score: {best_result.cross_val_mean:.4f} ± {best_result.cross_val_std:.4f}")

print(f"\n💾 Saved Artifacts:")
print(f"  📦 Best Model: {best_model_path.name}")
print(f"  📦 All Models: {all_models_path.name}")
print(f"  📋 Training Results: {results_summary_path.name}")
print(f"  📋 Model Metadata: {metadata_path.name}")
print(f"  📄 Evaluation Report: {Path(report_path).name}")

print(f"\n📊 Model Performance Summary:")
for model_name, result in sorted(training_results.items(), 
                               key=lambda x: x[1].test_accuracy, reverse=True):
    status = "🥇" if model_name == best_model_name else "  "
    print(f"  {status} {model_name}: {result.test_accuracy:.4f} accuracy")

print(f"\n🚀 Next Steps:")
print(f"  1️⃣ Deploy the best model using the model deployment pipeline")
print(f"  2️⃣ Set up model monitoring and performance tracking")
print(f"  3️⃣ Consider ensemble methods for improved performance")
print(f"  4️⃣ Implement automated retraining pipeline")
print(f"  5️⃣ Create inference endpoints for model serving")

print(f"\n✅ Model training pipeline completed successfully!")
print(f"📁 All artifacts saved to: {MODELS_DIR}")

logger.info(f"Model training pipeline completed successfully - Best: {best_model_name} ({best_result.test_accuracy:.4f})")

In [ ]:
# Clean up and final notes
plt.close('all')  # Close all matplotlib figures

print("Resources cleaned up")
print("Model training notebook completed successfully!")
print("\nRelated Notebooks:")
print("  01_getting_started.ipynb - Environment setup")
print("  02_data_processing_pipeline.ipynb - Data preparation")
print("  03_model_training.ipynb - Current notebook")
print("  04_vertex_ai_training.ipynb - Cloud training (Next)")
print("  05_model_deployment.ipynb - Model deployment (Next)")

print("\nFinal Summary:")
print(f"  Best Model: {best_model_name}")
print(f"  Test Accuracy: {best_result['test_accuracy']:.4f}")
print(f"  Training Time: {best_result['training_time']:.2f} seconds")
print(f"  Models Saved: {MODELS_DIR}")

print("\nModel training notebook completed successfully!")